# Gpt4All
---

`gpt4all` es una herramienta de código abierto que permite desplegar grandes modelos del lenguaje (LLMs) en cualquier hardware. Una de las mayores ventajas es que podemos utilizar modelos de hasta 13 miles de millones de parámetros en un computador personal o de escritorio sin la necesidad de tener una unidad de procesamiento gráfico (GPU - Graphical Processing Unit) y realizando la inferencia en un procesador convencional (CPU).

<center><img src="https://drive.google.com/uc?export=view&id=1bPqMcRD1sylQOppygNakQBx5p0Ewf2TZ" width="20%"></center>

Esta herramienta surge como una alternativa a:

- Herramientas que no dan un acceso directo a los modelos como OpenAI.
- Herramientas que requieren muchos recursos computacionales como tarjetas gráficas de mucha potencia.

Veamos cómo podemos realizar la instalación desde _Python_:

In [ ]:
!pip install gpt4all

Adicionalmente, utilizaremos algunas utilidades generales:

In [ ]:
from IPython.display import display

## **1. Búsqueda de Modelos**
---

`gpt4all` utiliza distintas estrategias para permitir utilizar modelos en CPU de forma eficiente. Veamos algunas de estas:

<center><img src="https://drive.google.com/uc?export=view&id=1iF3trBjj4cWdmzbdNZx1fJRu6_-DtFy_" width="50%"></center>

- **Modelos pequeños**: en `gpt4all` disponemos de modelos que no son tan masivos como los que ofrece OpenAI. Por ejemplo, el modelo detrás de ChatGPT (GPT-3.5) utiliza 175 billones de parámetros. En cambio, los modelos disponibles tienen entre 3 y 13 billones de parámetros.
- **Cuantificación**: los modelos de `gpt4all` realizan inferencia con técnicas de cuantificación, la cual trata de una técnica computacional que permite comprimir datos al reducir la precisión numérica. Por ejemplo, hay modelos que utilizan una versión cuantificada a 4 bits del modelo. Esto difiere de los modelos convencionales que trabajan con precisiones de 32 o 64 bits, lo que significa en una reducción de alrededor de 8 hasta 16 veces del uso de memoria de los modelos.
- **Optimización**: los modelos de `gpt4all` vienen listos para su uso en inferencia. Internamente fueron implementados en lenguajes de bajo nivel como C/C++ lo que permite que se ejecuten de una forma bastante eficiente y no requieren un framework intermedio como `pytorch` o `tensorflow`.

Para poder utilizar un modelo de `gpt4all` debemos identificar principalmente dos cosas:

- Tamaño del modelo.
- Conjunto de datos donde fue entrenado.

Esto lo podemos encontrar en la descripción en la página [oficial](https://gpt4all.io/index.html) como se puede observar en el siguiente video:

In [ ]:
#@markdown ##**Ejecute esta celda para ver el video.**
from IPython.display import IFrame
IFrame(
        src="https://drive.google.com/file/d/1CaJX9IJccJnygoA5EqhwAwfSsXHUWSw1/preview",
        width="768px",
        height="432px"
        )

Vamos a descargar un modelo `falcon` con cuantificación a 4 bits:

In [ ]:
# ![[ -d mlds7_models/ ]] && rm -rf mlds7_models
!git clone https://huggingface.co/juselara1/mlds7_models

Como podemos ver, el modelo se descarga con el nombre `ggml-model-gpt4all-falcon-q4_0.bin`:

In [ ]:
!ls mlds7_models/

## **2. Carga de Modelos**
---

Para cargar un modelo, debemos importar la clase `GPT4ALL`:

In [ ]:
from gpt4all import GPT4All

Para cargar el modelo, debemos indicar el nombre del archivo descargado:

> **Nota**: este modelo está cargado y utilizará los recursos del equipo donde estemos ejecutando este código. Esto es diferente de OpenAI donde la ejecución se realiza en un servidor de la misma empresa y únicamente tenemos acceso al resultado final.

In [ ]:
model = GPT4All(
        model_name="ggml-model-gpt4all-falcon-q4_0.bin",
        model_path="mlds7_models/"
        )
display(model)

Es importante resaltar que `gpt4all` está pensado fundamentalmente para inferencia. Veamos una comparativa de las tres herramientas que estamos presentando:

| Aspecto | HuggingFace | OpenAI | GPT4All |
| --- | --- | --- | --- |
| Entrenamiento | ✔ | ✔ | ✗ |
| Modificación de modelos | ✔ | ✗ | ✗ |
| Inferencia rápida | ✗ | ✔ | ✔ |
| Código abierto | ✔ | ✗ | ✔ |

Es importante resaltar que aunque OpenAI permite entrenamiento de modelos, nunca tendremos acceso a estos y únicamente podemos utilizarlos para inferencia tal y como los modelos que ya existen en esta plataforma.

Con `gpt4all`, principalmente estaremos realizando dos tareas: **generación de texto** y **sesiones de chat**.

## **3. Generación de Texto**
---

Cualquiera de los modelos de `gpt4all` puede usarse para la generación de texto a partir de una cadena original.

<center><img src="https://drive.google.com/uc?export=view&id=1rOfivYF0dQI9vvwA-etx1wsKxd0B9QEL" width="90%"></center>

Veamos cómo funciona esto para obtener una descripción de Alan Turing.

In [ ]:
text = "Alan Turing was "

Ahora, utilizamos el modelo para generar 10 tokens (determinado por el parámetro `max_tokens`):

In [ ]:
output = model.generate(text, max_tokens=30)
display(output)

El método `generate` permite utilizar los siguientes hiperparámetros:

- `max_tokens`: máximo número de tokens a generar.
- `temp`: temperatura para la predicción, son valores positivos en donde, entre más alto sea este valor, las respuestas son menos deterministas.
- `top_k`: específica un tamaño para seleccionar los tokens más probables a seleccionar en cada paso de la generación.
- `top_p`: equivalente a `top_k` con la diferencia que este hiperparámetro señala un valor de probabilidad acumulada que deben tener los tokens de la muestra.
- `repeat_penalty`: penalización al modelo por repetitividad.
- `repeat_last_n`: indica qué tantas palabras previas son consideradas para evaluar la repetitividad.
- `n_batch`: número de tokens que se procesan en paralelo.
- `streaming`: específica si el resultado se puede utilizar como un generador.

Veamos un ejemplo con un valor de temperatura mayor (menos determinístico), esto puede llevar al efecto de **alucinación** (inventarse cosas que parecen verdaderas):

In [ ]:
output = model.generate(text, temp=100.0, max_tokens=30)
print(output)

Veamos un ejemplo con un valor de temperatura más bajo (más determinístico, obtenemos respuestas parecidas cada vez que generamos texto):

In [ ]:
output = model.generate(text, temp=0.1, max_tokens=30)
display(output)

## **4. Sesiones de Chat**
---

`gpt4all` permite implementar de forma sencilla conversaciones por medio de las sesiones de chat. En especial, las sesiones de chat cumplen dos roles:

1. Al modelo se le asigna una plantilla (template) para que la generación sirva como un chat.
2. Salidas intermedias del modelo se almacenan en un cache para que el histórico de la conversación se representa directamente. Esto acelera la inferencia.

La sesión de chat se crea con el método `chat_session` dentro de un contexto. Veamos un ejemplo:

In [ ]:
with model.chat_session():
    response = model.generate("Hello", top_k=1)
    response = model.generate("Who invented the light bulb?", top_k=1)
    response = model.generate("Thanks", top_k=1)
    display(model.current_chat_session)

## **Recursos Adicionales**
---

Los siguientes enlaces corresponden a las sitios donde encontrará información útil para profundizar en los temas vistos en este taller guiado:

- [GPT4All in Python](https://docs.gpt4all.io/gpt4all_python.html#quickstart).
- [GPT4All: Running an Open-source ChatGPT Clone on Your Laptop](https://betterprogramming.pub/gpt4all-running-an-open-source-chatgpt-clone-on-your-laptop-71ebe8600c71).